# Загрузка данных

In [1]:
# !pip install kaggle

Нужен свой токен с kaggle

In [2]:
!echo '{"username":"","key":""}' > ../.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d datasnaek/youtube -p ../datasets --unzip

100%|██████████████████████████████████████| 55.9M/55.9M [00:15<00:00, 6.77MB/s]
100%|██████████████████████████████████████| 55.9M/55.9M [00:15<00:00, 3.72MB/s]


In [4]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local") \
    .config('spark.sql.autoBroadcastJoinThreshold', 0) \
    .config('spark.sql.adaptive.enabled', 'false') \
    .getOrCreate()

# Задание 1

scored_videos - датасет на основе файла USvideos.csv с добавлением колонки, содержащей скор (показатель качества) видео. Никто не знает, как считать скор, поэтому формулу предлагается придумать вам. Но она должна включать в себя просмотры, лайки, дизлайки видео, лайки и дизлайки к комментариям к этому видео.

In [5]:
videos = spark.read.option('header', 'true').option("inferSchema", "true").csv('../datasets/USvideos.csv')
videos.show(5)

+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|   video_id|               title|   channel_title|category_id|                tags|  views| likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+----------------+-----------+--------------------+-------+------+--------+-------------+--------------------+-----+
|XpVt6Z1Gjjo|1 YEAR OF VLOGGIN...|Logan Paul Vlogs|         24|logan paul vlog|l...|4394029|320053|    5931|        46245|https://i.ytimg.c...|13.09|
|K4wEI5zhHB0|iPhone X — Introd...|           Apple|         28|Apple|iPhone 10|i...|7860119|185853|   26679|            0|https://i.ytimg.c...|13.09|
|cLdxuaxaQwc|         My Response|       PewDiePie|         22|              [none]|5845909|576597|   39774|       170708|https://i.ytimg.c...|13.09|
|WYYvHb03Eog|Apple iPhone X fi...|       The Verge|         28|apple iphone x ha...|2642103| 24975| 

In [6]:
videos.count()

7998

In [7]:
comments_schema = T.StructType([
    T.StructField("video_id", T.StringType(), True),
    T.StructField("comment_text", T.StringType(), True),
    T.StructField("likes", T.IntegerType(), True),
    T.StructField("replies", T.IntegerType(), True)
])
comments = spark.read.option('header', 'true').option("mode", "DROPMALFORMED").schema(comments_schema).csv('../datasets/UScomments.csv')
comments.show(5)

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|XpVt6Z1Gjjo|Logan Paul it's y...|    4|      0|
|XpVt6Z1Gjjo|I've been followi...|    3|      0|
|XpVt6Z1Gjjo|Say hi to Kong an...|    3|      0|
|XpVt6Z1Gjjo| MY FAN . attendance|    3|      0|
|XpVt6Z1Gjjo|         trending 😉|    3|      0|
+-----------+--------------------+-----+-------+
only showing top 5 rows



In [8]:
comments.count()

691722

Предлагаю ранжировать видео через линейную комбинацию признаков. Поскольку нет меток для регрессии, веса для признаков выберу по интуиции. Так мы получим некое значение по степени "крутости". Поскольку у одного ролиа может быть несколько комментариев, для простоты можно посчитать агрегацию по лайкам и ответам, например, сумму

## $$w_1 * views + w_2 * likes + w_3 * dislikes + w_4 * sum(comments\_likes) + w_5 * sum(comments\_replies)$$

In [9]:
comments_agg = (
    comments
    .groupBy(F.col('video_id'))
    .agg(
        F.sum(F.col('likes')).alias('comments_likes'),
        F.sum(F.col('replies')).alias('comments_replies'),
    )
)

In [10]:
comments_agg.show(5)

+-----------+--------------+----------------+
|   video_id|comments_likes|comments_replies|
+-----------+--------------+----------------+
|xPS7bqBePSs|          1037|              28|
|dInwVhRtN4E|            63|              13|
|rn5Xgak1zzA|            14|               7|
|TzyraAp3jaY|          1126|              48|
|eHq6ZA6uKOg|           797|             138|
+-----------+--------------+----------------+
only showing top 5 rows



In [11]:
videos = (
    videos
    .join(comments_agg, on='video_id', how='left')
)

In [12]:
videos.select('video_id', 'title', 'views', 'likes', 'dislikes', 'comments_likes', 'comments_replies').show(5)

+-----------+--------------------+-------+------+--------+--------------+----------------+
|   video_id|               title|  views| likes|dislikes|comments_likes|comments_replies|
+-----------+--------------------+-------+------+--------+--------------+----------------+
|WYYvHb03Eog|Apple iPhone X fi...|2642103| 24975|    4542|           158|              16|
|K4wEI5zhHB0|iPhone X — Introd...|7860119|185853|   26679|          null|            null|
|sjlHnJvXdQs|   iPhone X (parody)|1168130| 96666|     568|           206|              17|
|cLdxuaxaQwc|         My Response|5845909|576597|   39774|           176|              70|
|XpVt6Z1Gjjo|1 YEAR OF VLOGGIN...|4394029|320053|    5931|           202|              24|
+-----------+--------------------+-------+------+--------+--------------+----------------+
only showing top 5 rows



In [13]:
@F.udf(returnType=T.FloatType())
def model_score(
    views,
    likes,
    dislikes,
    comments_likes,
    comments_replies,
    w_1: float = 0.1, 
    w_2: float = 10.0,
    w_3: float = -100.0,
    w_4: float = 2.0,
    w_5: float = 1.0,
):
    score = w_1 * views + w_2 * likes + w_3 * dislikes + w_4 * comments_likes + w_5 * comments_replies
    return score

подстелим соломку и обработаем пропуски

In [14]:
scored_videos = (
    videos
    .na.fill({'views': 0, 'likes': 0, 'dislikes': 0, 'comments_likes': 0, 'comments_replies': 0})
    .withColumn('model_score',
                model_score(F.col('views'), F.col('likes'), F.col('dislikes'), F.col('comments_likes'), F.col('comments_replies'))
               )
)

In [15]:
scored_videos.select('video_id', 'title', 'views', 'likes', 'dislikes', 'comments_likes', 'comments_replies', 'model_score').show(10)

+-----------+--------------------+-------+------+--------+--------------+----------------+-----------+
|   video_id|               title|  views| likes|dislikes|comments_likes|comments_replies|model_score|
+-----------+--------------------+-------+------+--------+--------------+----------------+-----------+
|WYYvHb03Eog|Apple iPhone X fi...|2642103| 24975|    4542|           158|              16|    60092.3|
|K4wEI5zhHB0|iPhone X — Introd...|7860119|185853|   26679|             0|               0|   -23358.1|
|sjlHnJvXdQs|   iPhone X (parody)|1168130| 96666|     568|           206|              17|  1027102.0|
|cLdxuaxaQwc|         My Response|5845909|576597|   39774|           176|              70|  2373583.0|
|zgLtEob6X-Q|Honest Trailers -...|1056891| 29943|     878|            48|               2|   317417.1|
|Ayb_2qbZHm4| Honest College Tour| 859289| 34485|     726|            64|              20|   358326.9|
|8wNr-NQImFg|The Check In: HUD...| 666169|  9985|     297|            51|

## Задание 2

categories_score - датасет по категориям, в котором присутствуют следующие поля: Название категории (не id, он непонятный для аналитиков!). Медиана показателя score из датасета scored_videos по каждой категории.

Для расчета медианы нельзя использовать встроенную Spark-функцию median из пакета pyspark.sql.functions

Сначала прочитаем категории, извлечем их названия, а потом подклеим по category_id к видео

In [16]:
category_schema = T.StructType([
    T.StructField("etag", T.StringType(), True),
    T.StructField("kind", T.StringType(), True),
    T.StructField("items", T.ArrayType(T.MapType(T.StringType(), T.StringType(), True), True)),
])

In [17]:
category = spark.read.option("multiline","true").json('../datasets/US_category_id.json', schema=category_schema)
category = category.withColumn('items', F.explode(F.col('items')))
category.show()

+--------------------+--------------------+--------------------+
|                etag|                kind|               items|
+--------------------+--------------------+--------------------+
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|youtube#videoCate...|{kind -> youtube#...|
|"m2yskBQFythfE4ir...|you

Достанем заголовок из вложения

In [18]:
snippet_schema = T.StructType([
    T.StructField("channelId", T.StringType(), True),
    T.StructField("title", T.StringType(), True),
    T.StructField("assignable", T.BooleanType(), True),
])

In [19]:
category = (
    category
    .withColumn('snippet_json',
                F.from_json(F.col('items.snippet'), schema=snippet_schema)
               )
    .select(
        F.col('items.id').alias('category_id'),
        F.col('snippet_json.title').alias('category_title')
    )
)

In [20]:
category.show()

+-----------+--------------------+
|category_id|      category_title|
+-----------+--------------------+
|          1|    Film & Animation|
|          2|    Autos & Vehicles|
|         10|               Music|
|         15|      Pets & Animals|
|         17|              Sports|
|         18|        Short Movies|
|         19|     Travel & Events|
|         20|              Gaming|
|         21|       Videoblogging|
|         22|      People & Blogs|
|         23|              Comedy|
|         24|       Entertainment|
|         25|     News & Politics|
|         26|       Howto & Style|
|         27|           Education|
|         28|Science & Technology|
|         29|Nonprofits & Acti...|
|         30|              Movies|
|         31|     Anime/Animation|
|         32|    Action/Adventure|
+-----------+--------------------+
only showing top 20 rows



In [21]:
category.count()

32

Возможно, тут лучше broadcast сделать, так как категорий мало

In [22]:
categories_score = (
    scored_videos
    .join(F.broadcast(category), on='category_id', how='left')
)

In [23]:
categories_score.explain()

== Physical Plan ==
*(6) Project [category_id#20, video_id#17, title#18, channel_title#19, tags#21, views#256, likes#257, dislikes#258, comment_total#25, thumbnail_link#26, date#27, comments_likes#259L, comments_replies#260L, model_score#275, category_title#377]
+- *(6) BroadcastHashJoin [category_id#20], [cast(category_id#376 as int)], LeftOuter, BuildRight, false
   :- *(6) Project [video_id#17, title#18, channel_title#19, category_id#20, tags#21, views#256, likes#257, dislikes#258, comment_total#25, thumbnail_link#26, date#27, comments_likes#259L, comments_replies#260L, pythonUDF0#415 AS model_score#275]
   :  +- BatchEvalPython [model_score(views#256, likes#257, dislikes#258, comments_likes#259L, comments_replies#260L)#274], [pythonUDF0#415]
   :     +- *(4) Project [video_id#17, title#18, channel_title#19, category_id#20, tags#21, coalesce(views#22, 0) AS views#256, coalesce(likes#23, 0) AS likes#257, coalesce(dislikes#24, 0) AS dislikes#258, comment_total#25, thumbnail_link#26, d

In [24]:
categories_score.select('video_id', 'category_id', 'category_title', 'model_score').show(10)

+-----------+-----------+--------------------+-----------+
|   video_id|category_id|      category_title|model_score|
+-----------+-----------+--------------------+-----------+
|WYYvHb03Eog|         28|Science & Technology|    60092.3|
|K4wEI5zhHB0|         28|Science & Technology|   -23358.1|
|sjlHnJvXdQs|         23|              Comedy|  1027102.0|
|cLdxuaxaQwc|         22|      People & Blogs|  2373583.0|
|zgLtEob6X-Q|          1|    Film & Animation|   317417.1|
|Ayb_2qbZHm4|         23|              Comedy|   358326.9|
|8wNr-NQImFg|         23|              Comedy|   136882.9|
|XpVt6Z1Gjjo|         24|       Entertainment|  3047261.0|
|_ANP3HR1jsM|         22|      People & Blogs|   790697.3|
|_HTXMhKWqnA|         28|Science & Technology|   695568.4|
+-----------+-----------+--------------------+-----------+
only showing top 10 rows



Теперь сгруппируем и посчитаем медиану по скорингу

In [25]:
import numpy as np
import pandas as pd

In [26]:
@F.pandas_udf(T.FloatType(), F.PandasUDFType.GROUPED_AGG)
def median(scores) -> float:
    return np.median(scores)

/usr/local/spark/python/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [27]:
categories_score = (
    categories_score
    .groupBy('category_title')
    .agg(
        median(F.col('model_score')).alias('median_model_score')
    )
)

In [28]:
categories_score.orderBy('median_model_score').show()

+--------------------+------------------+
|      category_title|median_model_score|
+--------------------+------------------+
|               Shows|           1363.25|
|Nonprofits & Acti...|         4697.8496|
|     News & Politics|            4839.5|
|              Sports|           20351.2|
|    Autos & Vehicles|          21810.05|
|              Gaming|           44072.0|
|    Film & Animation|           73552.0|
|       Entertainment|           81670.6|
|           Education|           81915.9|
|      Pets & Animals|           85878.1|
|Science & Technology|          89047.05|
|     Travel & Events|          95248.41|
|      People & Blogs|           95632.2|
|               Music|         107226.95|
|       Howto & Style|         125808.91|
|              Comedy|          215351.9|
+--------------------+------------------+



# Задание 3

In [29]:
import time
from tqdm import tqdm

popular_tags - датасет по самым популярным тэгам (название тэга + количество видео с этим тэгом). В исходном датасете тэги лежат строкой в поле tags. Другие разработчики уже сталкивались с подобной задачей, поэтому написали Scala-функцию для разбиения тегов. Но не доверяйте им вслепую! Обязательно напишите свою функцию разбиения строки на тэги и сравните время работы с её Scala-версией. Можно замерять своими силами, а можно воспользоваться библиотекой timeit.

In [30]:
from pyspark.sql.column import Column
from pyspark.sql.column import _to_java_column
from pyspark.sql.column import _to_seq

In [31]:
sc = spark.sparkContext

Посмотрим на Scala UDF

In [32]:
def scala_splitTags_udf_wrapper(tags):
    splitTags_idf = sc._jvm.CustomUDFs.splitTagsUDF()
    return Column(splitTags_idf.apply(_to_seq(sc, [tags], _to_java_column)))

In [33]:
scala_times = []
for _ in tqdm(range(1_000)):
    start_time = time.time()
    videos.withColumn('splitted_tags', scala_splitTags_udf_wrapper(F.col('tags'))).select('video_id', 'tags', 'splitted_tags').count()
    work_time = time.time() - start_time
    scala_times.append(work_time)

100%|██████████| 1000/1000 [01:17<00:00, 12.86it/s]


In [34]:
np.mean(scala_times), np.std(scala_times)

(0.07720073127746582, 0.015258046526695577)

Это было быстро...

Теперь напишем свою UDF

In [35]:
@F.pandas_udf(T.ArrayType(T.StringType(), True), F.PandasUDFType.SCALAR)
def split_tags_custom_udf(tags):
    return tags.split('|')

In [36]:
python_times = []
for _ in tqdm(range(1_000)):
    start_time = time.time()
    videos.withColumn('splitted_tags', split_tags_custom_udf(F.col('tags'))).select('video_id', 'tags', 'splitted_tags').count()
    work_time = time.time() - start_time
    python_times.append(work_time)

100%|██████████| 1000/1000 [01:06<00:00, 15.02it/s]


In [37]:
np.mean(python_times), np.std(python_times)

(0.06611887526512146, 0.005023053088385481)

По наблюдениям сложно сказать, какой вариант здесь значительно лучше, поскольку и тот и другой от итерации к итерации были быстрее. Возможно, при значительном объеме данных разница будет заметнее, но сейчас она мало на что влияет. Обычно, стандартное отклонение у pandas_udf повыше, нет уверенности, что данная реализация будет стабильна

Выберем вариант со Scala UDF, потому что так интереснее

In [38]:
tags = videos.withColumn('splitted_tags', scala_splitTags_udf_wrapper(F.col('tags'))).select('video_id', 'tags', 'splitted_tags')

In [39]:
tags.show()

+-----------+--------------------+--------------------+
|   video_id|                tags|       splitted_tags|
+-----------+--------------------+--------------------+
|XpVt6Z1Gjjo|logan paul vlog|l...|[logan paul vlog,...|
|K4wEI5zhHB0|Apple|iPhone 10|i...|[Apple, iPhone 10...|
|cLdxuaxaQwc|              [none]|            [[none]]|
|WYYvHb03Eog|apple iphone x ha...|[apple iphone x h...|
|sjlHnJvXdQs|jacksfilms|parody...|[jacksfilms, paro...|
|cMKX2tE5Luk|a24|a24 films|a24...|[a24, a24 films, ...|
|8wNr-NQImFg|Late night|Seth M...|[Late night, Seth...|
|_HTXMhKWqnA|iPhone X|iphone x...|[iPhone X, iphone...|
|_ANP3HR1jsM|Roman Atwood|Roma...|[Roman Atwood, Ro...|
|zgLtEob6X-Q|screenjunkies|scr...|[screenjunkies, s...|
|Ayb_2qbZHm4|Collegehumor|CH o...|[Collegehumor, CH...|
|CsdzflTXBVQ|best floyd maywea...|[best floyd maywe...|
|l864IBj7cgw|The Tonight Show|...|[The Tonight Show...|
|4MkC65emkG4|mtv|video|online|...|[mtv, video, onli...|
|vu_9muoxT50|America's Got Tal...|[America's Got

In [40]:
tags.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- splitted_tags: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [41]:
popular_tags = (
    tags
    .withColumn('tag', 
                F.explode(F.col('splitted_tags'))
               )
    .groupBy('tag')
    .agg(
        F.count(F.col('video_id')).alias('count_video_id')
    )
    .orderBy(F.desc('count_video_id'))
)

In [42]:
popular_tags.show()

+---------+--------------+
|      tag|count_video_id|
+---------+--------------+
|    funny|           722|
|   comedy|           572|
|   [none]|           491|
|     2017|           309|
|   how to|           284|
|     vlog|           273|
|    humor|           258|
|   makeup|           254|
|    music|           250|
| tutorial|           235|
|     food|           224|
|    video|           219|
|   review|           218|
|celebrity|           211|
|     news|           211|
|   beauty|           210|
|interview|           209|
|  science|           197|
|      Pop|           190|
|  trailer|           180|
+---------+--------------+
only showing top 20 rows



# Задание 4

И личная просьба от Марка: он любит котов (а кто не их не любит!) и хочет найти самые интересные комментарии (топ-5) к видео про котов. “Видео про котов” - видео, у которого есть тэг “cat”.

In [43]:
(
    tags
    .filter(
        F.array_contains(F.col('splitted_tags'), 'cat')
    )
).show()

+-----------+--------------------+--------------------+
|   video_id|                tags|       splitted_tags|
+-----------+--------------------+--------------------+
|Vjc459T6wX8|Maru|cat|kitty|pe...|[Maru, cat, kitty...|
|0Yhaei1S5oQ|SciShow|science|H...|[SciShow, science...|
|-1fzGnFwz9M|cartoon|simons ca...|[cartoon, simons ...|
|Vjc459T6wX8|Maru|cat|kitty|pe...|[Maru, cat, kitty...|
|0Yhaei1S5oQ|SciShow|science|H...|[SciShow, science...|
|Vjc459T6wX8|Maru|cat|kitty|pe...|[Maru, cat, kitty...|
|Vjc459T6wX8|Maru|cat|kitty|pe...|[Maru, cat, kitty...|
|BY3SLVNBkeo|cartoon|simons ca...|[cartoon, simons ...|
|7V1J_MDi9Lg|Husky's First How...|[Husky's First Ho...|
|BY3SLVNBkeo|cartoon|simons ca...|[cartoon, simons ...|
|7V1J_MDi9Lg|Husky's First How...|[Husky's First Ho...|
|BY3SLVNBkeo|cartoon|simons ca...|[cartoon, simons ...|
|7V1J_MDi9Lg|Husky's First How...|[Husky's First Ho...|
|BY3SLVNBkeo|cartoon|simons ca...|[cartoon, simons ...|
|7V1J_MDi9Lg|Husky's First How...|[Husky's First

In [44]:
spark.stop()